In [ ]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = 'plotly'
TEMPLATE = 'seaborn'

from sklearn.linear_model import LinearRegression

import util

# Lecture 22 – Pipelines, Generalization

## DSC 80, Winter 2023

### 📣 Announcements

- Project 4 is due on **Thursday, March 9th at 11:59PM**.
- Lab 8 (modeling) is due **today at 11:59PM**.
- Project 5 will be released within the next few days and will be due on **Thursday, March 23rd at 11:59PM**.
    - You _cannot_ use slip days on Project 5.
- RSVP to the Senior Capstone Showcase on March 15th at [**hdsishowcase.com**](https://hdsishowcase.com).
    - There is no live lecture for DSC 80 on the day of the showcase.

### Agenda

- The modeling process.
- Transformers in `sklearn`.
- Pipelines.
- Generalization.

## The modeling process

### The modeling process

<center><img src="imgs/image_0.png" width="60%"></center>

1. Create (engineer) features to best reflect the "meaning" behind data.

2. Choose a model that is appropriate to capture the relationships between features ($X$) and the target/response ($y$).

3. Select a loss function and fit the model (i.e., determine $w^*$).

4. Evaluate the model (e.g. using RMSE or $R^2$).

**We can perform all of the above directly in `sklearn`!**

### `preprocessing` and `linear_model`s

For the **feature engineering** step of the modeling pipeline, we will use `sklearn`'s [`preprocessing`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) module.

<center><img src="imgs/feature_part.png" width="30%"></center>

For the **model creation** step of the modeling pipeline, we will use `sklearn`'s [`linear_model`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model) module, as we've already seen. `linear_model.LinearRegression` is an example of an **estimator** class.

<center><img src="imgs/model_part.png" width="36%"></center>

## Transformers in `sklearn`

### Transformer classes

- **Transformers** take in "raw" data and output "processed" data. They are used for **creating features**.

- The input to a transformer should be a multi-dimensional `numpy` array.
    - Inputs can be DataFrames, but `sklearn` only looks at the values (i.e. it calls `to_numpy()` on input DataFrames).

- The output of a transformer is a `numpy` array (never a DataFrame or Series).

- Transformers, like most relevant features of `sklearn`, are **classes**, not functions, meaning you need to instantiate them and call their methods.

### Case study: Restaurant tips 🧑‍🍳

We'll continue working with our trusty `tips` dataset.

In [ ]:
tips = px.data.tips()
tips.head()

### Example transformer: `Binarizer`

The `Binarizer` transformer allows us to map a quantitative sequence to a sequence of 1s and 0s, depending on whether values are above or below a threshold.

|Property|Example|Description|
|---|---|---|
|Initialize with parameters| `binar = Binarizer(thresh)` | set x=1 if x > thresh, else 0|
|Transform data in a dataset | `feat = binar.transform(data)` | Binarize all columns in `data`|

First, we need to import the relevant class from `sklearn.preprocessing`. (Tip: import just the relevant classes you need from `sklearn`.)

In [ ]:
from sklearn.preprocessing import Binarizer

Let's try binarizing `'total_bill'`. We'll say a "large" bill is one that is **strictly** greater than \$20.

In [ ]:
tips['total_bill'].head()

First, we initialize a `Binarizer` object with the threshold we want.

In [ ]:
bi = Binarizer(threshold=20)

Then, we call `bi`'s `transform` method and pass it the data we'd like to transform. Note that its input and output are both 2D.

In [ ]:
transformed_bills = bi.transform(tips[['total_bill']]) # Must give transform a 2D array/DataFrame.
transformed_bills[:5]

### Example transformer: `StdScaler`

- `StdScaler` **standardizes** data using the mean and standard deviation of the data.

$$z(x_i) = \frac{x_i - \text{mean of } x}{\text{SD of } x}$$

- Unlike `Binarizer`, `StdScaler` **requires some knowledge (mean and SD) of the dataset before transforming**.

- As such, we need to **`fit`** an `StdScaler` transformer before we can use the `transform` method.

* Typical usage: fit transformer on a sample, use that fit transformer to transform future data.


### Example transformer: `StdScaler`

It only makes sense to standardize the already-quantitative features of `tips`, so let's select just those.

In [ ]:
tips_quant = tips[['total_bill', 'size']]
tips_quant.head()

Let's initialize a `StandardScaler` object.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
stdscaler = StandardScaler()

Note that the following **does not work!** The error message is very helpful.

In [ ]:
stdscaler.transform(tips_quant)

Instead, we need to first call the `fit` method on `stdscaler`.

In [ ]:
# This is like saying "determine the mean and SD of each column in tips_quant".
stdscaler.fit(tips_quant)

Now, `transform` will work.

In [ ]:
# First column is 'total_bill', second column is 'size'.
tips_quant_z = stdscaler.transform(tips_quant)
tips_quant_z[:5]

We can also access the mean and variance `stdscaler` computed for each column:

In [ ]:
stdscaler.mean_

In [ ]:
stdscaler.var_

Note that we can call `transform` on DataFrames other than `tips_quant`. We will do this often – fit a transformer on one dataset (training data) and use it to transform other datasets (test data).

In [ ]:
stdscaler.transform(tips_quant.sample(5))

### `StdScaler` summary

|Property|Example|Description|
|---|---|---|
|Initialize with parameters| `stdscaler = StandardScaler()` | z-score the data (no parameters) |
|Fit the transformer| `stdscaler.fit(X)` | Compute the mean and SD of `X`|
|Transform data in a dataset | `feat = stdscaler.transform(X_new)` | z-score `X_new` with mean and SD of `X`|

### Example transformer: `OneHotEncoder`

Let's keep just the categorical columns in `tips`.

In [ ]:
tips_cat = tips[['sex', 'smoker', 'day', 'time']]
tips_cat.head()

Like `StdScaler`, we will need to `fit` our `OneHotEncoder` transformer before it can transform anything.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder()
ohe.fit(tips_cat)

We can look at the unique values (i.e. categories) in each column by using the `categories_` attribute:

In [ ]:
ohe.categories_

In [ ]:
ohe.transform(tips_cat)

Since the resulting matrix is **sparse** – most of its elements are 0 – `sklearn` uses a more efficient representation than a regular `numpy` array. That's no issue, though:

In [ ]:
ohe.transform(tips_cat).toarray()

Notice that the column names from `tips_cat` are no longer stored anywhere (remember, `fit` converts the input to a `numpy` array before proceeding).

We can use the `get_feature_names` method on `ohe` to access the names of the one-hot-encoded columns, though:

In [ ]:
ohe.get_feature_names() # x0, x1, x2, and x3 correspond to column names in tips_cat.

In [ ]:
pd.DataFrame(ohe.transform(tips_cat).toarray(), 
             columns=ohe.get_feature_names()) # If we need a DataFrame back, for some reason.

## Pipelines

<center><img src="imgs/image_0.png" width="50%"></center>

<br>

So far, we've used transformers for feature engineering and models for prediction. We can combine these steps into a single `Pipeline`.

### `Pipeline`s in `sklearn`

- To instantiate a `Pipeline`, we must provide a list with zero or more transformers followed by a single model.
    - All "steps" must have `fit` methods, and all but the last must have `transform` methods.
    - Template: `pl = Pipeline([feat_trans1, feat_trans2, ..., mdl])`.
    
```py

```

- Once a `Pipeline` is instantiated, you can fit **all** steps (transformers and model) using a single call to the `fit` method.
```py
pl.fit(X, y)
```

- To make predictions using **raw, untransformed data**, use `pl.predict`.

- The actual list we provide `Pipeline` with must be a list of **tuples**, where
    - The first element is a "name" (that we choose) for the step.
    - The second element is a transformer or estimator instance.

### Our first `Pipeline`

Let's build a `Pipeline` that:
- One hot encodes the categorical features in `tips`.
- Fits a regression model on the one hot encoded data.

In [ ]:
tips_cat = tips[['sex', 'smoker', 'day', 'time']]
tips_cat.head()

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pl = Pipeline([
    ('one-hot', OneHotEncoder()),
    ('lin-reg', LinearRegression())
])

Now that `pl` is instantiated, we `fit` it the same way we would fit the individual steps.

In [ ]:
pl.fit(tips_cat, tips['tip'])

Now, to make predictions using **raw data**, all we need to do is use `pl.predict`:

In [ ]:
pl.predict([['Female', 'Yes', 'Sat', 'Lunch']])

In [ ]:
pl.predict(tips_cat.iloc[:5])

`pl` performs **both** feature transformation and prediction with just a single call to `predict`!

We can access individual "steps" of a `Pipeline` through the `named_steps` attribute:

In [ ]:
pl.named_steps

In [ ]:
pl.named_steps['one-hot'].transform(tips_cat).toarray()

In [ ]:
pl.named_steps['lin-reg'].coef_

`pl` also has a `score` method, the same way a fit `LinearRegression` instance does:

In [ ]:
pl.score(tips_cat, tips['tip'])

### More sophisticated `Pipeline`s

- In the previous example, we one hot encoded every input column. **What if we want to perform different transformations on different columns?**

- **Solution:** Use a `ColumnTransformer`.
    - Instantiate a `ColumnTransformer` using a list of tuples, where:
        - The first element is a "name" we choose for the transformer.
        - The second element is a transformer instance (e.g. `OneHotEncoder()`).
        - The third element is a **list of relevant column names**.

- `ColumnTransformer` is extremely useful, but it was only added to `sklearn` in 2018!

### Planning our first `ColumnTransformer`

In [ ]:
from sklearn.compose import ColumnTransformer

Let's perform different transformations on the quantitative and categorical features of `tips` (note that we are not transforming `'tip'`).

In [ ]:
tips_features = tips.drop('tip', axis=1)
tips_features.head()

- We will leave the `'total_bill'` column untouched.

- To the `'size'` column, we will apply the `Binarizer` transformer with a threshold of 2 (big tables vs. small tables).

- To the categorical columns, we will apply the `OneHotEncoder` transformer.

- In essence, we will create a transformer that reproduces the following DataFrame:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>size</th>
      <th>x0_Female</th>
      <th>x0_Male</th>
      <th>x1_No</th>
      <th>x1_Yes</th>
      <th>x2_Fri</th>
      <th>x2_Sat</th>
      <th>x2_Sun</th>
      <th>x2_Thur</th>
      <th>x3_Dinner</th>
      <th>x3_Lunch</th>
      <th>total_bill</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>16.99</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>10.34</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>21.01</td>
    </tr>
    <tr>
      <th>3</th>
      <td>0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>23.68</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>24.59</td>
    </tr>
  </tbody>
</table>


### Building a `Pipeline` using a `ColumnTransformer`

Let's start by creating our `ColumnTransformer`.

In [ ]:
preproc = ColumnTransformer(
    transformers=[
        ('size', Binarizer(threshold=2), ['size']),
        ('categorical_cols', OneHotEncoder(), ['sex', 'smoker', 'day', 'time'])
    ],
    remainder='passthrough' # Specify what to do with all other columns ('total_bill' here) – drop or passthrough.
)

Now, let's create a `Pipeline` using `preproc` as a transformer, and `fit` it:

In [ ]:
pl = Pipeline([
    ('preprocessor', preproc), 
    ('lin-reg', LinearRegression())
])

In [ ]:
pl.fit(tips_features, tips['tip'])

Prediction is as easy as calling `predict`:

In [ ]:
tips_features.head()

In [ ]:
# Note that we fit the Pipeline using tips_features, not tips_features.head()!
pl.predict(tips_features.head())

We can even call each transformer in `pl['preprocessor']` individually to re-create the transformed DataFrame. (There's no _practical_ reason to do this, it's more for illustration.)

In [ ]:
dfs = []
for trans in pl['preprocessor'].transformers_:
    if isinstance(trans[1], str) and trans[1] == 'passthrough':
        df = tips_features.iloc[:, trans[2]]
    else:
        vals = trans[1].transform(tips_features[trans[2]])
        columns = trans[2]
        if str(trans[1]) == 'OneHotEncoder()':
            vals = vals.toarray()
            columns = trans[1].get_feature_names()
        df = pd.DataFrame(vals, columns=columns)
    dfs.append(df)

pd.concat(dfs, axis=1)

### Aside: `FunctionTransformer`

A transformer you'll often use as part of a `ColumnTransformer` is the `FunctionTransformer`, which enables you to use your own functions on entire columns. Think of it as the `sklearn` equivalent of `apply`.

In [ ]:
from sklearn.preprocessing import FunctionTransformer

In [ ]:
f = FunctionTransformer(np.sqrt)
f.transform([1, 2, 3])

### Summary: `Pipeline`s

- `Pipeline`s are powerful because they allow you to perform feature engineering and training/prediction all through a single object.

- It's important to understand what each step of a `Pipeline` does. Neural networks work _similarly_ to `sklearn` `Pipeline`s, in that they follow a well-defined sequence of steps to make predictions.

## Generalization

### Motivation

- You and Billy are studying for an upcoming exam. You both decide to test your understanding by taking a **practice exam**.
    - Your logic: If you do well on the practice exam, you should do well on the real exam.

- You each take the practice exam once and look at the solutions afterwards.

- **Your strategy**: Memorize the answers to all practice exam questions, e.g. "Question 1: A; Question 2: C; Question 3: A."

- **Billy's strategy**: Learn high-level concepts from the solutions, e.g. "data are NMAR if the likelihood of missingness depends on the missing values themselves."

- Who will do better on the **practice exam**? Who will probably do better on the **real exam**? 🧐

### Evaluating the quality of a model

- So far, we've computed the RMSE (and $R^2$) of our fit regression models on the **data that we used to fit them**, i.e. the **training data**.

- We've said that Model A is **better** than Model B if Model A's RMSE is **lower** than Model B's RMSE.
    - Remember, our **training data** is a sample from the data generating process.
    - Just because a model fits the training data well doesn't mean it will **generalize** and work well on **similar, unseen samples**!

### Example: Overfitting and underfitting

Let's collect two samples $\{(x_i, y_i)\}$ from the same **data generating process**.

In [ ]:
np.random.seed(23) # For reproducibility.

def sample_dgp(n=100):
    x = np.linspace(-2, 3, n)
    y = x ** 3 + (np.random.normal(0, 3, size=n))
    return pd.DataFrame({'x': x, 'y': y})

sample_1 = sample_dgp()
sample_2 = sample_dgp()

For now, let's just look at Sample 1. The relationship between $x$ and $y$ is roughly **cubic**; that is, $y \approx x^3$ (remember, in reality, you won't get to see the DGP).

In [ ]:
px.scatter(sample_1, x='x', y='y', title='Sample 1', template=TEMPLATE)

### Polynomial regression

Let's fit three **polynomial** models on Sample 1:
- Degree 1.
- Degree 3.
- Degree 25.

The `PolynomialFeatures` transformer will be helpful here.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# fit_transform fits and transforms the same input.
d2 = PolynomialFeatures(3)
d2.fit_transform(np.array([1, 2, 3, 4, -2]).reshape(-1, 1))

Below, we look at our three models' predictions on Sample 1 (which they were trained on).

In [ ]:
# Look at the definition of train_and_plot in util.py if you're curious as to how the plotting works.
fig = util.train_and_plot(train_sample=sample_1, test_sample=sample_1, degs=[1, 3, 25])
fig.update_layout(title='Trained on Sample 1, Performance on Sample 1')

The degree 25 polynomial has the lowest RMSE on Sample 1.

How do the same fit polynomials look on Sample 2?

In [ ]:
fig = util.train_and_plot(train_sample=sample_1, test_sample=sample_2, degs=[1, 3, 25])
fig.update_layout(title='Trained on Sample 1, Performance on Sample 2')

- The degree 3 polynomial has the lowest RMSE on Sample 2. 


- Note that **we didn't get to see Sample 2 when fitting our models**! 


- As such, it seems that the degree 3 polynomial **generalizes better** to unseen data than the degree 25 polynomial does.

What if we fit a degree 1, degree 3, and degree 25 polynomial **on Sample 2** as well?

In [ ]:
util.plot_multiple_models(sample_1, sample_2, degs=[1, 3, 25])

**Key idea**: Degree 25 polynomials seem to **vary more when trained on different samples** than degree 3 and 1 polynomials do.

### Bias and variance

The training data we have access to is a sample from the DGP. We are concerned with our model's ability to **generalize** and work well on **different datasets** drawn from the same DGP.

Suppose we **fit** a model $H$ (e.g. a degree 3 polynomial) on **several different datasets** from a DGP. There are three sources of error that arise:

* ⭐️ **Bias**: **The expected deviation between a predicted value and an actual value**.
    - In other words, **for a given $x_i$, how far is $H(x_i)$ from the true $y_i$, on average?**
    - Low bias is good! ✅
    - High bias is a sign of **underfitting**, i.e. that our model is too **basic** to capture the relationship between our features and response.

- ⭐️ **Model variance ("variance")**: **The variance of a model's predictions**.
    - In other words, **for a given $x_i$, what is the variance of $H(x_i)$ across all datasets**?
    - Low model variance is good! ✅
    - High model variance is a sign of **overfitting**, i.e. that our model is too **complicated** and is prone to fitting to the noise in our training data.

- **Observation variance**: The variance due to the random noise in the process we are trying to model (e.g. measurement error). _We can't control this, without collecting more data!_

Here, suppose:
- The <span style='color:#c6283f'><b>red bulls-eye</b></span> represents your **true weight and height** 🧍.
- The <span style='color:#080c6f'><b>dark blue darts</b></span> represent **predictions of your weight and height** using different models that were fit on the same DGP. 
<br>

<center><img src="imgs/image_5.png" width="40%"></center>

We'd like our models to be in the top left, but in practice that's hard to achieve!

## Summary, next time

### Summary

- Pipelines in `sklearn` combine one or more transformers with a single model (estimator), allowing us to perform feature engineering and prediction through a single object.
- We want to build models that **generalize** well to unseen data.
    - Models that have **high bias** are too simple to represent complex relationships in data, and underfit.
    - Models that have **high variance** are overly complex for the relationships in the data, and vary a lot when fit on different datasets. Such models overfit to the training data.

### Next time

How do we choose the right model complexity, so that our model has the right "balance" between bias and variance?